Approach 1: Self-Taught

In [ ]:
%pip install -q "requests==2.32.4"
%pip install -q langchain-community deepinfra python-dotenv vaderSentiment transformers pandas OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import json
import re
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

from langchain_community.llms import DeepInfra
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from openai import OpenAI

In [ ]:
import pandas as pd

# URL of the CSV file
fp_csv_url = "https://media.githubusercontent.com/media/ttran293/cmsc-671-project/refs/heads/main/clean_dataset/FinancialPhraseBank/FinancialPhraseBank_train.csv"
imdb_csv_url = "https://media.githubusercontent.com/media/ttran293/cmsc-671-project/refs/heads/main/clean_dataset/IMDB/IMDB_train.csv"
mdr_csv_url = "https://media.githubusercontent.com/media/ttran293/cmsc-671-project/refs/heads/main/clean_dataset/MDR/MDR_train.csv"

fpb_df = pd.read_csv(fp_csv_url)
imdb_df = pd.read_csv(imdb_csv_url)
mdr_df = pd.read_csv(mdr_csv_url)

In [ ]:
from getpass import getpass
from huggingface_hub import login

In [ ]:
hf_token = getpass("Enter your Hugging Face token (input hidden): ")
login(hf_token)

Enter your Hugging Face token (input hidden): ··········


In [ ]:
DEEPINFRA_TOKEN = getpass("Enter your DeepInfra token (input hidden): ")

Enter your DeepInfra token (input hidden): ··········


In [ ]:
from openai import OpenAI

openai = OpenAI(
    api_key=DEEPINFRA_TOKEN,
    base_url="https://api.deepinfra.com/v1/openai",
)


def llm_response(prompt, max_token):
  chat_completion = openai.chat.completions.create(
      model="meta-llama/Meta-Llama-3.1-8B-Instruct",
      messages=[
          {"role": "user", "content": prompt},
      ],
      temperature=0.2,
      top_p=0.9,
      max_tokens=max_token,
      extra_body={
          "repetition_penalty": 1.2,
      },
  )

  return chat_completion.choices[0].message.content


In [ ]:
# llm = DeepInfra(model_id="meta-llama/Meta-Llama-3.1-8B-Instruct")
model = "siebert/sentiment-roberta-large-english"
classifier = pipeline("sentiment-analysis", model=model, truncation=True, max_length=512)
analyzer = SentimentIntensityAnalyzer()

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def estimate_tokens(text):
    words = len(text.split())
    return int(words * 1.3)

def get_preserve_prompt(sentence):
    prompt = f"""
    You are a text summarization assistant.
    Your only job is to output a JSON object.

    Task:
    Summarize the input text in the original tone

    Input text (do NOT modify or ignore any part of it):
    \"\"\"{sentence}\"\"\"

    Requirements:
    - Do NOT add new information or opinions.
    - Do NOT explain your reasoning.
    - Output MUST be valid JSON (double quotes, no comments, no trailing commas).
    - Do NOT include markdown or any text before or after the JSON.

    Respond ONLY with a JSON object of this exact shape:
    {{
      "result": "your summary here"
    }}
    """
    return prompt


def get_no_preserve_prompt(sentence):
    prompt = f"""
    You are a text summarization assistant.
    Your only job is to output a JSON object.

    Task:
    Summarize the input text in a neutral tone

    Input text (do NOT modify or ignore any part of it):
    \"\"\"{sentence}\"\"\"

    Requirements:
    - Do NOT add new information or opinions.
    - Do NOT explain your reasoning.
    - Output MUST be valid JSON (double quotes, no comments, no trailing commas).
    - Do NOT include markdown or any text before or after the JSON.

    Respond ONLY with a JSON object of this exact shape:
    {{
      "result": "your summary here"
    }}
    """
    return prompt

def get_summary_prompt(sentence):
    prompt = f"""
    You are a text summarization assistant.
    Your only job is to output a JSON object.

    Task:
    Summarize the input text

    Input text (do NOT modify or ignore any part of it):
    \"\"\"{sentence}\"\"\"

    Requirements:
    - Do NOT add new information or opinions.
    - Do NOT explain your reasoning.
    - Output MUST be valid JSON (double quotes, no comments, no trailing commas).
    - Do NOT include markdown or any text before or after the JSON.

    Respond ONLY with a JSON object of this exact shape:
    {{
      "result": "your summary here"
    }}
    """
    return prompt

def get_vader_sentiment(sentence):
    scores = analyzer.polarity_scores(sentence)
    compound = scores["compound"]
    if compound >= 0.05:
        label = "positive"
    elif compound <= -0.05:
        label = "negative"
    else:
        label = "neutral"
    return label, compound

def get_roberta_sentiment(sentence):
    result = classifier(sentence)
    return result[0]['label'], result[0]['score']

def decide_mode(sentence):
    _, score = get_roberta_sentiment(sentence)
    if score >= 0.7:
        return "preserve"
    return "neutral"

def validate_summary(mode, orig_label, orig_score, sum_label, sum_score, sum_vader_label, sum_vader_score):
    if mode == "neutral":
        if sum_label in ("NEUTRAL") or sum_vader_label in ("neutral"):
            return True
        return False

    if mode == "preserve":
        if orig_label in ("POSITIVE", "NEGATIVE"):
            if sum_label != orig_label:
                return False
            if sum_score < 0.5:
                return False
        return True

    return True

#old
def extract_result_from_raw(raw, fallback):
    raw_clean = str(raw).strip()

    raw_no_fence = re.sub(r"```.*?```", "", raw_clean, flags=re.DOTALL)

    try:
        data = json.loads(raw_no_fence)
        if isinstance(data, dict) and "result" in data:
            return str(data["result"]).strip()
    except Exception:
        pass

    candidate_objs = re.findall(r'\{[^{}]*"result"[^{}]*\}', raw_no_fence, flags=re.DOTALL)

    last_good = None
    for obj in candidate_objs:
        try:
            data = json.loads(obj)
            if isinstance(data, dict) and "result" in data:
                last_good = str(data["result"]).strip()
        except Exception:
            continue

    if last_good is not None:
        return last_good

    m = re.findall(r'"result"\s*:\s*"([^"]+)"', raw_no_fence)
    if m:
        return m[-1].strip()

    return fallback

#new
import json
import re
def extract_result_from_content(string):
    text = string.strip()

    if text.startswith("```"):
        text = re.sub(r"^```(?:json)?", "", text, flags=re.IGNORECASE).strip()
        text = re.sub(r"```$", "", text).strip()

    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in model output")

    json_str = text[start:end + 1]

    data = json.loads(json_str)
    return data.get("result", "")

def extract_result_loose(string):
    s = string.strip()

    start = s.find("{")
    end = s.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("No JSON-like object found")

    block = s[start:end + 1]

    m = re.search(r'"result"\s*:\s*"(.*)"\s*}', block, re.DOTALL)
    if not m:
        raise ValueError("No `result` field found")

    val = m.group(1)

    if val.startswith('"') and val.endswith('"'):
        val = val[1:-1]

    return val

def clean_outer_quotes(s):
    s = s.strip()

    if s.startswith(r"\"") and s.endswith(r"\""):
        return s[2:-2].strip()

    if s.startswith('"') and s.endswith('"'):
        return s[1:-1].strip()

    return s

def extract_result_robust(raw_output, fallback_sentence):
    try:
        res = extract_result_from_content(raw_output)
        if res:
            return res.strip()
    except Exception:
        pass

    try:
        res = extract_result_loose(raw_output)
        if res:
            return res.strip()
    except Exception:
        pass

    try:
        res = extract_result_from_raw(raw_output, fallback_sentence)
        if res:
            return res.strip()
    except Exception:
        pass

    return fallback_sentence.strip()

In [ ]:
def generate_summary(df, mode_label):
    results = []
    total = len(df)
    for idx, row in df.iterrows():
        print(f"Processing {idx+1}/{total}...")
        sentence = row['sentence']

        if mode_label == 'auto':
            effective_mode = decide_mode(sentence)
        else:
            effective_mode = mode_label

        if effective_mode == 'preserve':
            prompt = get_preserve_prompt(sentence)
        elif effective_mode == 'neutral':
            prompt = get_no_preserve_prompt(sentence)
        else:
            prompt = get_summary_prompt(sentence)

        max_tokens = estimate_tokens(sentence) * 3 + 200

        try:
            raw = llm_response(prompt, max_tokens)
        except Exception as e:
            raw = ""
            summarized_text = sentence
        else:
            ext_result = extract_result_robust(raw, sentence)
            summarized_text = clean_outer_quotes(ext_result)
            if not summarized_text:
                summarized_text = sentence

        original_sentiment, original_score = get_roberta_sentiment(sentence)
        summarized_roberta_sentiment, summarized_roberta_score = get_roberta_sentiment(summarized_text)
        original_vader_sentiment, original_vader_score = get_vader_sentiment(sentence)
        summarized_vader_sentiment, summarized_vader_score = get_vader_sentiment(summarized_text)
        is_valid = validate_summary(effective_mode, original_sentiment, original_score, summarized_roberta_sentiment, summarized_roberta_score, summarized_vader_sentiment, summarized_vader_score)

        results.append({
            'original_text': sentence,
            'vader_original_sentiment': original_vader_sentiment,
            'vader_original_score': original_vader_score,
            'roberta_original_sentiment': original_sentiment,
            'roberta_original_score': original_score,
            'summarized_text': summarized_text,
            'vader_summarized_sentiment': summarized_vader_sentiment,
            'vader_summarized_score': summarized_vader_score,
            'roberta_summarized_sentiment': summarized_roberta_sentiment,
            'roberta_summarized_score': summarized_roberta_score,
            'mode_used': effective_mode,
            'is_valid': is_valid,
        })

    return results


In [ ]:
# fpb_df = fpb_df.head(5)
# imdb_df = imdb_df.head(5)
# mdr_df = mdr_df.head(5)

all_results = []

print("Processing FinancialPhraseBank...")
fpb_results = generate_summary(fpb_df, 'neutral')
all_results.extend(fpb_results)

print("\nProcessing IMDB...")
imdb_results = generate_summary(imdb_df, 'auto')
all_results.extend(imdb_results)

print("\nProcessing MDR...")
mdr_results = generate_summary(mdr_df, 'preserve')
all_results.extend(mdr_results)

# print(all_results)
result_df = pd.DataFrame(all_results)
import pandas as pd
from google.colab import files

result_df.to_csv('result.csv', index=False,encoding="utf-8-sig")
files.download('result.csv')

Streaming output truncated to the last 5000 lines.
Processing 1003/3000...
Processing 1004/3000...
Processing 1005/3000...
Processing 1006/3000...
Processing 1007/3000...
Processing 1008/3000...
Processing 1009/3000...
Processing 1010/3000...
Processing 1011/3000...
Processing 1012/3000...
Processing 1013/3000...
Processing 1014/3000...
Processing 1015/3000...
Processing 1016/3000...
Processing 1017/3000...
Processing 1018/3000...
Processing 1019/3000...
Processing 1020/3000...
Processing 1021/3000...
Processing 1022/3000...
Processing 1023/3000...
Processing 1024/3000...
Processing 1025/3000...
Processing 1026/3000...
Processing 1027/3000...
Processing 1028/3000...
Processing 1029/3000...
Processing 1030/3000...
Processing 1031/3000...
Processing 1032/3000...
Processing 1033/3000...
Processing 1034/3000...
Processing 1035/3000...
Processing 1036/3000...
Processing 1037/3000...
Processing 1038/3000...
Processing 1039/3000...
Processing 1040/3000...
Processing 1041/3000...
Processing 10

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>